Christine Plumejeaud-Perreau, UMR MIGRINTER 7301

07 avril 2022
projet ANR PORTIC

# Traitement des données de raw_flows pour regroupement par destination et analyse des produits et tonnages expédiés

In [3]:
import pandas as pd
import pandas.core.frame as pf

import json, urllib
import plotly as py
import pandas as pd
import numpy as np
import plotly.graph_objects as go


In [15]:
# Récupérer les raw_flows
#http://data.portic.fr/api/rawflows/?format=csv&date=1789&params=departure_uhgs_id,departure_fr,destination_uhgs_id,destination_fr,destination_partner_balance_supp_1789,destination_substate_1789_fr,destination_state_1789_fr,tonnage,outdate_fixed,commodity_purpose,commodity_purpose2,commodity_purpose3,commodity_purpose4,all_cargos,nb_cargo

#curl -o ./raw_flows_1789.csv "data.portic.fr/api/rawflows/?format=csv&date=1789&params=departure_uhgs_id,departure_fr,destination_uhgs_id,destination_fr,destination_partner_balance_supp_1789,destination_substate_1789_fr,destination_state_1789_fr,tonnage,outdate_fixed,commodity_purpose,commodity_purpose2,commodity_purpose3,commodity_purpose4,all_cargos,nb_cargo"
raw_flows = pd.read_csv("./raw_flows_1789.csv", encoding='utf8', sep=',')


type(raw_flows) #list

#df_dix_ans_smog = pd.DataFrame(dix_ans_smog, columns=['destination', '1781', '1782', '1785', '1786', '1787', '1788', '1789', '1790', '1791', 'Facade'])
print(raw_flows.shape) #(10081, 16)

print(raw_flows.columns)

''' 'Unnamed: 0', 'departure_uhgs_id', 'departure_fr',
       'destination_uhgs_id', 'destination_fr',
       'destination_partner_balance_supp_1789', 'destination_substate_1789_fr',
       'destination_state_1789_fr', 'tonnage', 'outdate_fixed',
       'commodity_purpose', 'commodity_purpose2', 'commodity_purpose3',
       'commodity_purpose4', 'all_cargos', 'nb_cargo' '''

(10081, 16)
Index(['Unnamed: 0', 'departure_uhgs_id', 'departure_fr',
       'destination_uhgs_id', 'destination_fr',
       'destination_partner_balance_supp_1789', 'destination_substate_1789_fr',
       'destination_state_1789_fr', 'tonnage', 'outdate_fixed',
       'commodity_purpose', 'commodity_purpose2', 'commodity_purpose3',
       'commodity_purpose4', 'all_cargos', 'nb_cargo'],
      dtype='object')


" 'Unnamed: 0', 'departure_uhgs_id', 'departure_fr',\n       'destination_uhgs_id', 'destination_fr',\n       'destination_partner_balance_supp_1789', 'destination_substate_1789_fr',\n       'destination_state_1789_fr', 'tonnage', 'outdate_fixed',\n       'commodity_purpose', 'commodity_purpose2', 'commodity_purpose3',\n       'commodity_purpose4', 'all_cargos', 'nb_cargo' "

    'departure_uhgs_id', 'departure_fr',
       'destination_uhgs_id', 'destination_fr', 'destination_substate_1789_fr',
       'destination_state_1789_fr', 'tonnage', 'outdate_fixed',
       'commodity_purpose', 'commodity_purpose2', 'commodity_purpose3',
       'commodity_purpose4', 'all_cargos', 'nbproduits'

In [132]:
# Filtrer

dunkerque = raw_flows.query('departure_fr=="Dunkerque"')
print(dunkerque.shape)
dunkerque = dunkerque.query('destination_partner_balance_supp_1789=="Etranger"') #Pas la France
dunkerque = dunkerque.query('destination_state_1789_fr!="Grande-Bretagne"') #Pas le smogglage vers l'angleterre
dunkerque = dunkerque.query('destination_uhgs_id!="C0000009"') #Afrique
dunkerque = dunkerque.query('destination_uhgs_id!="A0146289"') #Islande
dunkerque = dunkerque.query('destination_uhgs_id!="B0000715"')  #Terre-Neuve
dunkerque = dunkerque.query('commodity_purpose!="Lège"')  #pas à vide (Lège)
dunkerque = dunkerque.query('commodity_purpose!="lest"')  #pas à vide (lest)
dunkerque = dunkerque.query('commodity_purpose2!="lest"')  #pas à vide (lest)
dunkerque = dunkerque.query('commodity_purpose!="pêche"')  #pas pour la pêche
print(dunkerque.shape) #(193, 16)


print(dunkerque.columns)


dunkerque = dunkerque.drop(['Unnamed: 0', 'all_cargos'], axis=1)

print(dunkerque.shape) #(193, 14)
dunkerque.to_csv (r'./flux_dunkerque.csv', index = None)


(2539, 16)
(191, 16)
Index(['Unnamed: 0', 'departure_uhgs_id', 'departure_fr',
       'destination_uhgs_id', 'destination_fr',
       'destination_partner_balance_supp_1789', 'destination_substate_1789_fr',
       'destination_state_1789_fr', 'tonnage', 'outdate_fixed',
       'commodity_purpose', 'commodity_purpose2', 'commodity_purpose3',
       'commodity_purpose4', 'all_cargos', 'nb_cargo'],
      dtype='object')
(191, 14)


## Premier apercu des destinations d'export de Dunkerque vers l'étranger, hors GB, hors pêche, hors navires à vide. 

In [29]:


gdata = pf.DataFrame({'count' : dunkerque.groupby( [ "destination_state_1789_fr","destination_uhgs_id"] ).size()}).reset_index()
print(gdata)

pf.DataFrame(dunkerque.groupby([ "destination_state_1789_fr","destination_uhgs_id"])['tonnage'].agg(['sum','count'])).reset_index()


   destination_state_1789_fr destination_uhgs_id  count
0                   Autriche            A0041886     17
1                   Autriche            A0046492      1
2                   Autriche            A0049808     56
3                      Brême            A0691896      1
4                   Danemark            A0912818      1
5                   Danemark            A1964976      5
6                    Espagne            A0079352      7
7                    Espagne            B0000056      1
8      Etats-Unis d'Amérique            B2827324      1
9                   Hambourg            A0743522      3
10                  Portugal            A0360085      1
11                  Portugal            A0364551      1
12                  Portugal            A0374720      6
13           Provinces-Unies            A0618784      4
14           Provinces-Unies            A0620777     10
15           Provinces-Unies            A0622825      1
16           Provinces-Unies            A0624785

,destination_state_1789_fr,destination_uhgs_id,sum,count
0,Autriche,A0041886,516.0,17
1,Autriche,A0046492,60.0,1
2,Autriche,A0049808,3570.0,56
3,Brême,A0691896,70.0,1
4,Danemark,A0912818,35.0,1
5,Danemark,A1964976,134.0,5
6,Espagne,A0079352,943.0,7
7,Espagne,B0000056,120.0,1
8,Etats-Unis d'Amérique,B2827324,350.0,1
9,Hambourg,A0743522,435.0,3


In [24]:
import csv
import json

by_pointcalls = []


for _, row in dunkerque.iterrows():
  if row['all_cargos'] != '':
    truc = row['all_cargos']
    #print(truc)
    truc = truc.replace("'", '"').replace("None", 'null')
    #.replace("bœ\x9c\x9cuf", "boeuf")
    cargos = json.loads(truc)
    #print(type(cargos))
    for cargo in cargos:
      if  cargo['commodity_standardized_fr'] != 'Lest' : 
        if  cargo['commodity_standardized_fr'] != None : 
          by_pointcalls.append({
            "cargo": cargo['commodity_purpose'],
            "cargo_standardized_fr": cargo['commodity_standardized_fr'],
            **row
          })
        else :
          by_pointcalls.append({
            "cargo": cargo['commodity_purpose'],
            "cargo_standardized_fr": cargo['commodity_purpose'].lower(),
            **row
          })
  else:
    by_pointcalls.append({
      "cargo": "",
      "cargo_standardized_fr": "",
      **row
    })

#

In [26]:
exports_dunkerque = pd.DataFrame(by_pointcalls, columns=['cargo', 'cargo_standardized_fr']+dunkerque.columns.tolist())

exports_dunkerque = exports_dunkerque.drop(['Unnamed: 0', 'commodity_purpose', 'commodity_purpose2', 'commodity_purpose3', 'commodity_purpose4', 'all_cargos'], axis=1)

print(exports_dunkerque.columns)
print(exports_dunkerque.shape) #(331, 12)
exports_dunkerque.to_csv (r'./exports_dunkerque.csv', index = None)
#exports_dunkerque

Index(['cargo', 'cargo_standardized_fr', 'departure_uhgs_id', 'departure_fr',
       'destination_uhgs_id', 'destination_fr',
       'destination_partner_balance_supp_1789', 'destination_substate_1789_fr',
       'destination_state_1789_fr', 'tonnage', 'outdate_fixed', 'nb_cargo'],
      dtype='object')
(331, 12)


In [135]:
import pandas as pd
import pandas.core.frame as pf

# Lire le fichier 
exports_dunkerque = pd.read_csv("./exports_dunkerque.csv")
exports_dunkerque['tonnage_per_product'] = exports_dunkerque['tonnage'] / exports_dunkerque['nb_cargo']

gdata = pf.DataFrame({'count' : exports_dunkerque.groupby( [ "destination_state_1789_fr","cargo_standardized_fr"] ).size()}).reset_index()
#print(gdata)

pd.DataFrame(exports_dunkerque.groupby([ "destination_state_1789_fr","cargo_standardized_fr"])['tonnage'].agg(['sum','count', 'mean'])).reset_index()

exports = pd.DataFrame(exports_dunkerque.groupby([ "destination_state_1789_fr","cargo_standardized_fr"])['tonnage','tonnage_per_product'].agg(['count', 'sum'])).reset_index()
exports.set_axis(['destination_state_1789_fr', 'cargo_standardized_fr', 'nb_ships', 'sum_tonnage', 'nb_ships2', 'sum_tonnage_per_product'], axis=1, inplace=True)
exports.to_csv (r'./exports_dunkerque_par_pays_par_produits.csv', index = None)



C:\Users\cplume01\AppData\Local\Temp\ipykernel_38568\4061757245.py:13: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.



In [136]:
# Lire le fichier 
#exports = pd.read_csv("./exports_dunkerque_par_pays_par_produits.csv")

#https://stackoverflow.com/questions/30631841/pandas-how-do-i-assign-values-based-on-multiple-conditions-for-existing-columns
conditions = [
    exports['cargo_standardized_fr'].apply(str.lower).isin(['avoine','orge', 'farines', 'farine', 'lin', 'blé', 'bled']) | exports['cargo_standardized_fr'].str.contains('lin'),
    exports['cargo_standardized_fr'].eq('Café') | exports['cargo_standardized_fr'].isin(['Café', 'Sucre', 'tabac restant de la cargaison', 'cottes de tabac', 'tabac faisant restant de son chargement']),
    exports['cargo_standardized_fr'].apply(str.lower).isin(['vin', 'eau-de-vie', 'genevre', 'taffias', 'taffia']),
    exports['cargo_standardized_fr'].apply(str.lower).isin(['charbon', 'bois']),
    exports['cargo_standardized_fr'].apply(str.lower).isin(['coton', 'mousseline']),
    exports['cargo_standardized_fr'].isin(['Peaux de boeuf', 'Laine', 'toile', 'cuir', 'cuirs']) ,
    exports['cargo_standardized_fr'].isin(['Diverses marchandises', 'panier', 'ozier', 'balloteries', 'cloux', 'huille de térbantine','marbre', 'marbres', 'restant de son chargement']),
    exports['cargo_standardized_fr'].isin(['Fruits', 'Fèves', 'raisins', 'figues', 'saumon','saumons' ]) ,
    exports['cargo_standardized_fr'].eq('Huile') ,	
]
#{"bled","avoine","orge","Avoine","farines","lin","blé"}'
choices = ['gold','coral', 'purple', 'black', 'lightblue', 'azure', 'lightgreen', 'pink', 'salmon']

exports['colors'] = np.select(conditions, choices, default='grey')

pd.DataFrame.sort_values(exports, by=['colors'], inplace=True)


codes, uniques = pd.factorize(exports['destination_state_1789_fr'])
codes_p, uniques_p = pd.factorize(exports['cargo_standardized_fr'])

print(codes)
print(uniques)

print(codes_p)
print(uniques_p)

exports['source'] =  codes_p
exports['target'] =  codes+len(uniques_p)
exports['retarget'] =  codes+len(uniques_p)+len(uniques)

exports


[ 0  1  2  2  2  2  3  4  2  5  2  1  1  2  1  1  0  0  4  2  1  0  1  0
  1  1  1  6  7  8  1  9  2 10  2  2  2  9  2  7  1  1  1  2  2  2  1  0
  0  9  4 10  3  6  2  2 10  1  2 10  8 10  2  6  4  2  1  2 10  1  4  0
  1  2]
Index(['Espagne', 'Provinces-Unies', 'Autriche', 'Etats-Unis d'Amérique',
       'Portugal', 'Brême', 'Hambourg', 'République de Gênes', 'Suède',
       'Toscane', 'Danemark'],
      dtype='object')
[ 0  1  2  3  4  1  5  6  7  8  9  7 10 10 11  8 12 13 14 15 13 16 16 17
 18 19 20 20 20 21 22 20 23 20 24 20 25 26 26 26 27 28 29 30 31 32 26 26
 33 34 26 26 26 26 35 36 37 38 39 40 41 41 42 43 41 43 43 44 45 42 42 41
 46 46]
Index(['toile', 'cuir', 'Laine', 'Peaux de boeuf', 'cuirs', 'Charbon', 'Bois',
       'Café', 'cottes de tabac', 'tabac faisant restant de son chargement',
       'Sucre', 'tabac restant de la cargaison', 'Farine', 'Orge',
       'mousseline', 'Graines de lin', 'Blé', 'Avoine', 'Futailles vides',
       'azur', 'tabac',
       'restant de sa car

,destination_state_1789_fr,cargo_standardized_fr,nb_ships,sum_tonnage,nb_ships2,sum_tonnage_per_product,colors,source,target,retarget
37,Espagne,toile,1,120.0,1,40.000000,azure,0,47,58
61,Provinces-Unies,cuir,1,50.0,1,25.000000,azure,1,48,59
7,Autriche,Laine,2,86.0,2,43.000000,azure,2,49,60
8,Autriche,Peaux de boeuf,1,20.0,1,10.000000,azure,3,49,60
13,Autriche,cuirs,3,145.0,3,48.333333,azure,4,49,60
...,...,...,...,...,...,...,...,...,...,...
51,Provinces-Unies,Eau-de-vie,1,270.0,1,270.000000,purple,42,48,59
45,Portugal,Eau-de-vie,2,430.0,2,280.000000,purple,42,51,62
36,Espagne,genevre,1,120.0,1,40.000000,purple,41,47,58
54,Provinces-Unies,Huile,1,50.0,1,16.666667,salmon,46,48,59


## Dessiner un diagramme alluvial montrant la répartition des produits expédiés de Dunkerque par destination

In [134]:
#from https://chart-studio.plotly.com/~alishobeiri/1591/plotly-sankey-diagrams/#/ 
#https://medium.com/plotly/4-interactive-sankey-diagram-made-in-python-3057b9ee8616
#https://stackoverflow.com/questions/59845407/plotly-express-vs-altair-vega-lite-for-interactive-plots
# https://matplotlib.org/3.1.0/tutorials/colors/colors.html
#https://towardsdatascience.com/sankey-diagram-basics-with-pythons-plotly-7a13d557401a 
#https://chart-studio.plotly.com/~alishobeiri/1591/plotly-sankey-diagrams/#/
#https://errorsfixing.com/plotly-sankey-is-it-able-to-put-the-node-labels-to-the-right-for-last-layercolumn/ 

import json, urllib
import plotly as py
import pandas as pd
import numpy as np
import plotly.graph_objects as go


print(exports.shape)#(74, 10)



fig =  go.Figure(data=[go.Sankey(
    node = dict(
      pad = 15,
      thickness = 30,
      line = dict(color = "black", width = 0.5),
      label = uniques_p.tolist()+uniques.tolist(),
      color = 'grey'
      #color = exports['colors'].dropna(axis=0, how='any')
    ),
    link = dict(
      source = exports['source'],
      target = exports['target'],
      value = exports['sum_tonnage_per_product'].dropna(axis=0, how='any'),
      color = exports['colors'].dropna(axis=0, how='any'),
  )
)])




fig.update_layout(title_text="Export des produits depuis Dunkerque vers l'étranger (hors Grande-Bretagne) en 1789 - (hors pêche)", font_size=10, height = 1000, width=700)
fig.show()

import plotly.io as pio

pio.write_image(fig, './sankey_export_product_Dunkerque_1789.png', format='png', scale=1.5, height=1400, width=700)



(74, 10)


In [125]:
# Un exemple de code, on s'en fout. 

data = dict(
type='sankey',
node = dict(
  pad = 15,
  thickness = 20,
  line = dict(
    color = "black",
    width = 0.5
  ),
  label = ["A1", "A2", "B1", "B2", "C1", "C2"],

  color = ["blue", "blue", "blue", "blue", 
  "gray", "white"]),#attempt to make it less visible

link = dict(
  source = [0,1,0,2,3,3],
  target = [2,3,3,4,4,5],
  value = [8,4,2,8,4,2],
  # attempt to add labels
  label= [8,4,2,8,4,2]))


import plotly.offline as py
layout =  dict(
  title = "Basic Sankey Diagram",
  font = dict(
    size = 10
  ),
    annotations=[
        dict(
            x=0.25,
            y=0.75,
            text='8',
            showarrow=False
        ),
        dict(
            x=0.75,
            y=0.25,
            text='4',
            showarrow=False
        )
    ]
 )

fig = dict(data=[data], layout=layout)
py.iplot(fig, validate=False)

## Export par pays et par produit

In [138]:

# Export par pays et par produit (faire une carte chroplèthe serait alors possible)

exports2 = pd.DataFrame(exports_dunkerque.groupby([ "destination_uhgs_id","cargo_standardized_fr"])['tonnage','tonnage_per_product'].agg(['count', 'sum'])).reset_index()
exports2.set_axis(['destination_state_1789_fr', 'cargo_standardized_fr', 'nb_ships', 'sum_tonnage', 'nb_ships2', 'sum_tonnage_per_product'], axis=1, inplace=True)
print(exports2.columns)
exports2.set_axis(['destination_state_1789_fr', 'cargo_standardized_fr', 'nb_ships', 'sum_tonnage', 'nb_ships2', 'sum_tonnage_per_product'], axis=1)
exports2.to_csv (r'./exports_dunkerque_par_pays_par_produits.csv', index = None)






Index(['destination_state_1789_fr', 'cargo_standardized_fr', 'nb_ships',
       'sum_tonnage', 'nb_ships2', 'sum_tonnage_per_product'],
      dtype='object')


C:\Users\cplume01\AppData\Local\Temp\ipykernel_38568\1813640315.py:1: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.



# Fabriquer un fichier résumé par port et détaillant les arrivées par produits sur chaque ligne. 

In [150]:
#https://stackoverflow.com/questions/30631841/pandas-how-do-i-assign-values-based-on-multiple-conditions-for-existing-columns

import pandas as pd
import pandas.core.frame as pf

# Lire le fichier 
exports_dunkerque = pd.read_csv("./exports_dunkerque.csv")
exports_dunkerque['tonnage_per_product'] = exports_dunkerque['tonnage'] / exports_dunkerque['nb_cargo']

print(exports_dunkerque.columns)
# Calculer le détail par produit

#Regroupement en catégories ad-hoc
conditions = [
    #exports_dunkerque['cargo_standardized_fr'] is not None, 
    exports_dunkerque['cargo_standardized_fr'].apply(str.lower).isin(['avoine','orge', 'farines', 'farine', 'lin', 'blé', 'bled']) 
    | exports['cargo_standardized_fr'].str.contains('lin'),
    exports_dunkerque['cargo_standardized_fr'].eq('Café') | exports['cargo_standardized_fr'].isin(['Café', 'Sucre', 'coton', 'coton', 'mousseline', 'tabac restant de la cargaison', 'cottes de tabac', 'tabac faisant restant de son chargement']),
    exports_dunkerque['cargo_standardized_fr'].apply(str.lower).isin(['vin', 'eau-de-vie', 'genevre', 'taffias', 'taffia']),
    exports_dunkerque['cargo_standardized_fr'].apply(str.lower).isin(['charbon', 'bois']),
    exports_dunkerque['cargo_standardized_fr'].isin(['Peaux de boeuf', 'Laine', 'toile', 'cuir', 'cuirs']) ,
    exports_dunkerque['cargo_standardized_fr'].isin(['Diverses marchandises', 'panier', 'ozier', 'balloteries', 'cloux', 'marbre', 'marbres','restant de son chargement']),
    exports_dunkerque['cargo_standardized_fr'].isin([ 'Fruits', 'Fèves', 'Huile', 'raisins', 'figues', 'saumon','saumons']) 
]

choices = [ 'cereales','produits coloniaux', 'alcools', 'chauffage',  'tissus', 'divers', 'alimentation']

exports_dunkerque['categorie'] = np.select(conditions, choices, default='sans categorie')

#Regrouper par destination et catégorie et agréger sur tonnage et tonnage_per_product
exports_tomap = pd.DataFrame(exports_dunkerque.groupby([ "destination_uhgs_id","categorie"])['tonnage', 'tonnage_per_product'].agg(['count', 'sum'])).reset_index()
exports_tomap.set_axis(['destination_uhgs_id', 'categorie', 'nb_ships', 'sum_tonnage', 'nb_ships2', 'sum_tonnage_per_product'], axis=1, inplace=True)
exports_tomap.drop('nb_ships2', axis=1, inplace=True)
exports_tomap = exports_tomap.round(1)
exports_tomap.to_csv (r'./exports_dunkerque_par_port_par_categorie_produits_bad.csv', index = None)

# Unstacker les produits : un par colonne
exports_tomap.set_index(["destination_uhgs_id", "categorie"], inplace=True)
cat_nb_ships = ['nb_ships_'+item for item in (choices+['sans categorie'])]
cat_sum_tonnage = ['sum_tonnage_'+item for item in (choices+['sans categorie'])]
cat_sum_tonnage_per_product = ['sum_tonnage_per_product_'+item for item in (choices+['sans categorie'])]
exports_tomap = exports_tomap.unstack().fillna(0)
col = cat_nb_ships + cat_sum_tonnage + cat_sum_tonnage_per_product
#print (col)
exports_tomap.set_axis(col, axis=1, inplace=True)
exports_tomap = exports_tomap.reset_index(level='destination_uhgs_id', col_level=1)
print(exports_tomap.shape) #(23, 25)

#Calculer le total par port
total = pd.DataFrame(exports_dunkerque.groupby([ "destination_uhgs_id"])['tonnage', 'tonnage_per_product'].agg(['count', 'sum'])).reset_index()
total.set_axis(['destination_uhgs_id', 'nb_ships_total', 'sum_tonnage_total', 'nb_ships2', 'sum_tonnage_per_product_total'], axis=1, inplace=True)
total.drop('nb_ships2', axis=1, inplace=True)
print(total.shape) #(23, 4)

# Fusionner les 2 dataframes
exports_tomap = pd.merge(
    exports_tomap,
    total,
    how="inner",
    left_on='destination_uhgs_id',
    right_on='destination_uhgs_id',
    sort=True,
    indicator=False)
exports_tomap['departure_uhgs_id'] = 'A0204180'
exports_tomap['departure_fr'] = 'Dunkerque'

print(exports_tomap.shape)#(23, 30)

exports_tomap

exports_tomap.to_csv (r'./exports_dunkerque_par_port_par_categorie_produits.csv', index = None)

Index(['cargo', 'cargo_standardized_fr', 'departure_uhgs_id', 'departure_fr',
       'destination_uhgs_id', 'destination_fr',
       'destination_partner_balance_supp_1789', 'destination_substate_1789_fr',
       'destination_state_1789_fr', 'tonnage', 'outdate_fixed', 'nb_cargo',
       'tonnage_per_product'],
      dtype='object')
(23, 25)
(23, 4)
(23, 30)


C:\Users\cplume01\AppData\Local\Temp\ipykernel_38568\2672930657.py:31: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.

C:\Users\cplume01\AppData\Local\Temp\ipykernel_38568\2672930657.py:50: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.



## Faire une carte avec VegaLite et Altair

In [151]:
#Pré-requis
#pip install altair vega_datasets panel altair_saver bokeh jupyter_bokeh

# D'abord vers la jointure avec les données géographiques

# Aller chercher les données de ports via l'API (à rajouter dans le load_data.sh)
#curl -o data/navigo_ports.csv "data.portic.fr/api/ports/?format=csv&srid=4326"
ports = pd.read_csv("http://data.portic.fr/api/ports/?format=csv&srid=4326", encoding='utf8')

# Fusionner les 2 dataframes
exports_tomap = pd.merge(
    exports_tomap,
    ports,
    how="inner",
    left_on='destination_uhgs_id',
    right_on='uhgs_id',
    sort=True,
    indicator=False)




Index(['destination_uhgs_id', 'nb_ships_cereales',
       'nb_ships_produits coloniaux', 'nb_ships_alcools', 'nb_ships_chauffage',
       'nb_ships_tissus', 'nb_ships_divers', 'nb_ships_alimentation',
       'nb_ships_sans categorie', 'sum_tonnage_cereales',
       'sum_tonnage_produits coloniaux', 'sum_tonnage_alcools',
       'sum_tonnage_chauffage', 'sum_tonnage_tissus', 'sum_tonnage_divers',
       'sum_tonnage_alimentation', 'sum_tonnage_sans categorie',
       'sum_tonnage_per_product_cereales',
       'sum_tonnage_per_product_produits coloniaux',
       'sum_tonnage_per_product_alcools', 'sum_tonnage_per_product_chauffage',
       'sum_tonnage_per_product_tissus', 'sum_tonnage_per_product_divers',
       'sum_tonnage_per_product_alimentation',
       'sum_tonnage_per_product_sans categorie', 'nb_ships_total',
       'sum_tonnage_total', 'sum_tonnage_per_product_total',
       'departure_uhgs_id', 'departure_fr'],
      dtype='object')
Index(['destination_uhgs_id', 'nb_ships_cere

In [171]:
#https://vega.github.io/vega/docs/projections/
#https://github.com/vega/vega-lite/issues/3488#issuecomment-386199585

import altair as alt
import panel as pn
pn.extension('vega')

#pays_url = './word1789_topojson.json'
pays_url = 'https://raw.githubusercontent.com/cplumejeaud/intoZcarto/main/python/word1789_topojson.json'
pays = alt.topo_feature(pays_url, 'word1789_geojson')

print(exports_tomap.columns)


variable = 'sum_tonnage_total' #'nb_ships_total', 'sum_tonnage_total', 'nb_ships2', 'sum_tonnage_per_product_total'
# World 1789, center on England
map = alt.layer(
    # use the sphere of the Earth as the base layer
    alt.Chart({'sphere': True}).mark_geoshape(
        fill='#e6f3ff'
    ),
    # add a graticule for geographic reference lines
    alt.Chart({'graticule': True}).mark_geoshape(
        stroke='#ffffff', strokeWidth=1
    ),
    # and then the countries of the world
    alt.Chart(pays).mark_geoshape(
        fill='lightgray', stroke='white', strokeWidth=0.5
    ),
    #the ports 
    alt.Chart(exports_tomap).mark_circle().encode(
        longitude='x:Q',
        latitude='y:Q',
        size=alt.Size('{}:Q'.format(variable), title='Analyse de la variable :  {}'.format(variable), scale=alt.Scale(domain=[-1, 7000], range=[10, 400], clamp=True, zero= False)), 
        color=alt.value('orange'),
        tooltip=['toponyme_standard_fr:N','{}:Q'.format(variable)]
    )
).properties(
    width=500,
    height=600,
    title='Exports de Dunkerque à l\'Etranger en 1789'
).project('conicEquidistant', scale=1000).configure_projection(center = [-5, 47]).configure_view(stroke=None)

map.save('map.json')
map.save('map_export_Dunkerque.html')

map


Index(['destination_uhgs_id', 'nb_ships_cereales',
       'nb_ships_produits coloniaux', 'nb_ships_alcools', 'nb_ships_chauffage',
       'nb_ships_tissus', 'nb_ships_divers', 'nb_ships_alimentation',
       'nb_ships_sans categorie', 'sum_tonnage_cereales',
       'sum_tonnage_produits coloniaux', 'sum_tonnage_alcools',
       'sum_tonnage_chauffage', 'sum_tonnage_tissus', 'sum_tonnage_divers',
       'sum_tonnage_alimentation', 'sum_tonnage_sans categorie',
       'sum_tonnage_per_product_cereales',
       'sum_tonnage_per_product_produits coloniaux',
       'sum_tonnage_per_product_alcools', 'sum_tonnage_per_product_chauffage',
       'sum_tonnage_per_product_tissus', 'sum_tonnage_per_product_divers',
       'sum_tonnage_per_product_alimentation',
       'sum_tonnage_per_product_sans categorie', 'nb_ships_total',
       'sum_tonnage_total', 'sum_tonnage_per_product_total',
       'departure_uhgs_id', 'departure_fr', 'Unnamed: 0', 'ogc_fid', 'uhgs_id',
       'x', 'y', 'total', 'topo

alt.LayerChart(...)


Christine Plumejeaud-Perreau, UMR MIGRINTER 7301
07 avril 2022
projet ANR PORTIC
